In [1]:
%load_ext jupyter_black

In [2]:
from enum import IntEnum, Flag, EnumMeta, auto
import pandas as pd


class WXCodeMeta(EnumMeta):
    def __iter__(self):
        for member in super().__iter__():
            yield member.name, member.value


class WXCodes(IntEnum, metaclass=WXCodeMeta):
    RA = 1
    TS = 2
    BR = 3
    SN = 4
    FG = 5


s = pd.Series(["RA", "FG"])

print(s)

print(s.replace(dict(WXCodes)))

0    RA
1    FG
dtype: object
0    1
1    5
dtype: int64


In [3]:
import urllib.parse
from enum import Enum, EnumMeta, auto
from typing import Iterator, TypeVar, Iterable, Generator, NewType
from collections import ChainMap


Names = TypeVar("Names", str, list[str], tuple[str])


def _urlencode(items: Iterable["QueryEnum"]):
    return urllib.parse.urlencode(
        tuple((member.__query_name__, member.value) for member in items)
    )


class QueryMap(ChainMap):
    def __str__(self) -> str:
        return _urlencode(super().__iter__())

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({', '.join(i.__repr__() for i in super().__iter__()) })"


StrGenerator = NewType("Generator[str, ...]", Generator[str, None, None])


class QueryEnumMeta(EnumMeta):
    def __values__(self) -> StrGenerator:
        yield from (member.value for member in super().__iter__())

    def __names__(self) -> StrGenerator:
        yield from (member.name for member in super().__iter__())

    def __string_contains__(self, __o: str) -> bool:
        return __o in self.__values__() or __o in self.__names__()

    def __str__(cls) -> str:
        return _urlencode(super().__iter__())

    def __getitem__(self, names: Names) -> QueryMap:
        if not isinstance(names, (list, tuple)):
            names = (names,)
        return QueryMap(tuple(self._member_map_[name] for name in names))

    def __contains__(self, __o: object) -> bool:
        if isinstance(__o, str):
            return self.__string_contains__(__o)
        return super().__contains__(__o)

    def __eq__(self, __o: object) -> bool:
        if isinstance(__o, str):
            return self.__string_contains__(__o)
        return super().__eq__(__o)

    def __hash__(self) -> int:
        return super().__hash__()

    def keys(self):
        return tuple(self.__names__())

    def values(self):
        return tuple(self.__values__())


class QueryEnum(str, Enum, metaclass=QueryEnumMeta):
    def _generate_next_value_(name: str, *_) -> str:
        return name

    def __repr__(self):
        return f"{self.__query_name__}={self.value}"

    @classmethod
    @property
    def __query_name__(cls) -> str:
        return cls.__name__.lower()


class Models(QueryEnum):
    GALWEM = auto()
    NAM = auto()
    GFS = auto()
    WRF_17K = "WRF-1.7k"


class Parameters(QueryEnum):
    wind_direction = auto()
    wind_speed = auto()
    wind_gust = auto()
    visibility = auto()
    present_wx = auto()
    ten_meter_temp = "10_m_temp"


localhost = "http://localhost:8080"
assert Parameters("10_m_temp") == Parameters.ten_meter_temp

assert (
    f"{localhost}?{Models[['GALWEM', 'NAM']]}"
    == f"{localhost}?{Models['GALWEM', 'NAM']}"
)

assert (
    f"{localhost}?{QueryMap(Parameters,Models)}" == f"{localhost}?{Models}&{Parameters}"
)

assert all(("GALWEM" in Models, Models.GALWEM in Models, Models in ("GALWEM", "NAM")))

assert Parameters.ten_meter_temp.upper() == "10_M_TEMP"

f"{localhost}?{Models['GALWEM']}&{Parameters}", f"{localhost}?{Models[['GALWEM', 'NAM']]}", f"{localhost}?{QueryMap(Models,Parameters)}",
Models["GALWEM"], QueryMap(Models, Parameters)

(QueryMap(models=GALWEM),
 QueryMap(parameters=wind_direction, parameters=wind_speed, parameters=wind_gust, parameters=visibility, parameters=present_wx, parameters=10_m_temp, models=GALWEM, models=NAM, models=GFS, models=WRF-1.7k))

In [4]:
import numpy as np

a = np.array(Models, dtype=object)
mask = (a == "GALWEM") | (a == "NAM")
f"{localhost}?{QueryMap(a[mask])}"

'http://localhost:8080?models=GALWEM&models=NAM'

In [5]:
import pandas as pd

s = pd.Series(tuple(Parameters), index=Parameters.keys())
f"{localhost}?{QueryMap(s[s.str.contains('wind')])}"

'http://localhost:8080?parameters=wind_direction&parameters=wind_speed&parameters=wind_gust'

In [6]:
def f(params: Parameters):
    return Parameters[params]


f(["wind_direction", "wind_speed"])

QueryMap(parameters=wind_direction, parameters=wind_speed)